### keras-retinanet 패키지를 이용하여 이미지와 영상 Object Detection 수행
*  Pretrained된 coco 모델을 로드 하고 이를 이용하여 Object Detection 수행

#### 관련 모듈 import 

In [73]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import base64
import codecs, json
import pickle
import matplotlib.pyplot as plt
import tensorflow.keras
from PIL import Image
%matplotlib inline


# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
#from keras_retinanet.utils.gpu import setup_gpu
from tensorflow.keras.models import Model


In [58]:
import tensorflow as tf

In [59]:
print(tf.__version__)

1.15.2


In [60]:
!pip install --ignore-installed --upgrade tensorflow-gpu==1.15.2

  Using cached tensorflow_gpu-1.15.2-cp36-cp36m-manylinux2010_x86_64.whl (411.0 MB)
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached protobuf-3.17.0-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached grpcio-1.37.1-cp36-cp36m-manylinux2014_x86_64.whl (4.2 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl
  Using cached gast-0.2.2-py3-none-any.whl
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached opt_einsum-3.3.0-py3-none-

In [61]:
!pip install keras==2.3.0

#### gpu 환경 설정

In [62]:
# 현 버전에서 이슈가 있어서 주석 처리합니다. 2020.08.29
#gpu = 0
#setup_gpu(gpu)

#### keras-retinanet으로 pretrained된 coco 모델 다운로드하고 해당 모델을 로드

In [63]:
# 아래 모델은 https://github.com/fizyr/keras-retinanet/releases 에서 download 받을 수 있음. 
# 해당 모델 h5 파일을 snapshot 디렉토리에 저장 후 retina model의 load_model()을 이용하여 모델 로딩.
# !wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5 

In [64]:
os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

'./keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'

In [65]:
model_path = os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

# pretrained coco 모델 파일을 retinanet 모델로 로딩.  
retina_model = models.load_model(model_path, backbone_name='resnet50')

tracking <tf.Variable 'Variable_10:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_11:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_12:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_13:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_14:0' shape=(9, 4) dtype=float32> anchors


/home/bottae/anaconda3/envs/tf115/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Feature Map 출력

In [66]:
output = retina_model.layers[-2].get_weights()
retina_model.predict_on_batch()

TypeError: predict_on_batch() missing 1 required positional argument: 'x'

#### coco 데이터 세트의 클래스id별 클래스명 지정. 

In [ ]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorbike',4:'aeroplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',10:'fire hydrant',
                        11:'stop sign',12:'parking meter',13:'bench',14:'bird',15:'cat',16:'dog',17:'horse',18:'sheep',19:'cow',20:'elephant',
                        21:'bear',22:'zebra',23:'giraffe',24:'backpack',25:'umbrella',26:'handbag',27:'tie',28:'suitcase',29:'frisbee',30:'skis',
                        31:'snowboard',32:'sports ball',33:'kite',34:'baseball bat',35:'baseball glove',36:'skateboard',37:'surfboard',38:'tennis racket',39:'bottle',40:'wine glass',
                        41:'cup',42:'fork',43:'knife',44:'spoon',45:'bowl',46:'banana',47:'apple',48:'sandwich',49:'orange',50:'broccoli',
                        51:'carrot',52:'hot dog',53:'pizza',54:'donut',55:'cake',56:'chair',57:'sofa',58:'pottedplant',59:'bed',60:'diningtable',
                        61:'toilet',62:'tvmonitor',63:'laptop',64:'mouse',65:'remote',66:'keyboard',67:'cell phone',68:'microwave',69:'oven',70:'toaster',
                        71:'sink',72:'refrigerator',73:'book',74:'clock',75:'vase',76:'scissors',77:'teddy bear',78:'hair drier',79:'toothbrush' 
                      }

In [ ]:
retina_model.summary()

In [ ]:
retina_model.layers[-1]

In [ ]:
# load image
imagePath = "beatles01.jpg"
image = read_image_bgr('../../data/image/'+imagePath)
print('image shape:', image.shape)
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# 모델에 입력전에 이미지 사전 처리. keras-retinanet은 image
image = preprocess_image(image)
image, scale = resize_image(image)
print('resized image size:', image.shape, 'scale:', scale)

image_arr = [[image]]

In [ ]:
# 중간 레이어의 output을 만들어 내는 모델 만들기
extract_model_P6 = keras.models.Model(inputs=retina_model.input, outputs=retina_model.get_layer('P6').output)

extract_model_P3 = keras.models.Model(inputs=retina_model.input, outputs=retina_model.get_layer('P3').output)
extract_model_P4 = keras.models.Model(inputs=retina_model.input, outputs=retina_model.get_layer('P4').output)
extract_model_P5 = keras.models.Model(inputs=retina_model.input, outputs=retina_model.get_layer('P5').output)
extract_model_P7 = keras.models.Model(inputs=retina_model.input, outputs=retina_model.get_layer('P7').output)

extract_features_by_layer_P6 = extract_model_P6.predict_on_batch(image_arr)

extract_features_by_layer_P3 = extract_model_P3.predict_on_batch(image_arr)
extract_features_by_layer_P4 = extract_model_P4.predict_on_batch(image_arr)
extract_features_by_layer_P5 = extract_model_P5.predict_on_batch(image_arr)
extract_features_by_layer_P7 = extract_model_P6.predict_on_batch(image_arr)

# 피쳐맵 차원 출력|
print(extract_features_by_layer.shape)

plt.figure(figsize=(16,8))

# 피쳐맵 차원에 맞게 출력 갯수 설정
#for i in range(extract_features_by_layer.shape[-1]):
for i in range(32):
    plt.subplot(4,8, i+1)
    
    plt.axis('off')
    plt.matshow(extract_features_by_layer_P6[0,:,:,i], cmap='gray', fignum=0)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))

for i in range(32):
    plt.subplot(4,8, i+1)
    
    plt.axis('off')
    plt.matshow(extract_features_by_layer_P3[0,:,:,i], cmap='gray', fignum=0)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))

for i in range(32):
    plt.subplot(4,8, i+1)
    
    plt.axis('off')
    plt.matshow(extract_features_by_layer_P4[0,:,:,i], cmap='gray', fignum=0)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))

for i in range(32):
    plt.subplot(4,8, i+1)
    
    plt.axis('off')
    plt.matshow(extract_features_by_layer_P5[0,:,:,i], cmap='gray', fignum=0)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))

for i in range(32):
    plt.subplot(4,8, i+1)
    
    plt.axis('off')
    plt.matshow(extract_features_by_layer_P7[0,:,:,i], cmap='gray', fignum=0)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
#for i in range(extract_features_by_layer.shape[-1]):
for i in range(32):
    plt.subplot(4,8, i+1)
    
    plt.axis('off')
    plt.matshow(extract_features_by_layer[0,:,:,i], cmap='gray', fignum=0)
plt.tight_layout()
plt.show

In [ ]:
#print(extract_features_by_layer)
print(len(extract_features_by_layer[0][0][0][0]))
print(extract_features_by_layer[0])

In [ ]:
#print(len(predict_feature[2][0]))
#predict_feature[2][0]
np_predict_feature = np.array(extract_features_by_layer[0][0])
print(np.shape(np_predict_feature))
np_predict_feature = np.reshape(np_predict_feature,(256,-1), order='C')
print(np.shape(np_predict_feature))

In [ ]:
#plt.figure(figsize=(300,200))
plt.matshow(np_predict_feature)
plt.show()

#### 원본 이미지를 Object Detection 후에 시각화 표시. 

In [84]:
# load image
imagePath = "beatles01.jpg"
image = read_image_bgr('../../data/image/'+imagePath)
print('image shape:', image.shape)
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)


# 모델에 입력전에 이미지 사전 처리. keras-retinanet은 image
image = preprocess_image(image)
image, scale = resize_image(image)
print('resized image size:', image.shape, 'scale:', scale)




# 이미지에 대해 Object Detection 수행. 
start = time.time()
boxes, scores, labels = retina_model.predict_on_batch(np.expand_dims(image, axis=0))
print(boxes.shape, scores.shape, labels.shape)
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# JSON 출력 결과를 저장할 배열
JsonObject = []


# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break
    
    b = box.astype(int)
    
    # JSON 출력 내용 입력
    JsonObject.append(
            {
                "objectID":int(label),
                "location":{
                    "xmin":float(b[0]),
                    "ymin":float(b[1]),
                    "xmax":float(b[2]),
                    "ymax":float(b[3])
                },
                "tag":str(labels_to_names_seq[label]),
                "reducedVector":["수납장 객체의 feature map"],
                "vectorShape":["feature map의 shape"]
            })
        
    
    color = label_color(label)

   
    
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
    draw_caption(draw, b, caption)

#JsonObject = JsonObject.tolist()

#print(JsonObject)

JSON_dict={"object":JsonObject,"IMG_URL":"이미지URL"}

print(JSON_dict)
 

# 배열을JSON 파일로 dumps
# indent=4로 설정하여 보기 좋게 만듬
# JSON 파일명은 경로명 + .json(파일 서식)
textFile = imagePath + ".json"
textJSON = json.dump(JSON_dict,codecs.open(textFile, 'w', encoding='utf-8'), indent=4)
with open(textFile, 'w') as f:
    f.write(textJSON)

plt.figure(figsize=(14, 14))
plt.axis('off')
plt.imshow(draw)
plt.show()

image shape: (633, 806, 3)
resized image size: (800, 1019, 3) scale: 1.263823064770932
(1, 300, 4) (1, 300) (1, 300)
processing time:  2.182638168334961
{'object': [{'objectID': 0, 'location': {'xmin': 228.0, 'ymin': 262.0, 'xmax': 374.0, 'ymax': 548.0}, 'tag': 'person', 'reducedVector': ['수납장 객체의 feature map'], 'vectorShape': ['feature map의 shape']}, {'objectID': 0, 'location': {'xmin': 558.0, 'ymin': 259.0, 'xmax': 721.0, 'ymax': 567.0}, 'tag': 'person', 'reducedVector': ['수납장 객체의 feature map'], 'vectorShape': ['feature map의 shape']}, {'objectID': 0, 'location': {'xmin': 49.0, 'ymin': 254.0, 'xmax': 198.0, 'ymax': 564.0}, 'tag': 'person', 'reducedVector': ['수납장 객체의 feature map'], 'vectorShape': ['feature map의 shape']}, {'objectID': 2, 'location': {'xmin': 140.0, 'ymin': 246.0, 'xmax': 253.0, 'ymax': 342.0}, 'tag': 'car', 'reducedVector': ['수납장 객체의 feature map'], 'vectorShape': ['feature map의 shape']}, {'objectID': 0, 'location': {'xmin': 380.0, 'ymin': 276.0, 'xmax': 553.0, 'ymax': 5

TypeError: Object of type 'ndarray' is not JSON serializable

In [ ]:
retina_model.summary()

In [ ]:
def get_detected_image_retina(model, img_array, use_copied_array, is_print=True):
    
    # copy to draw on
    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    img_array = preprocess_image(img_array)
    img_array, scale = resize_image(img_array)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img_array, axis=0))
    if is_print:
        print("object detection 처리 시간: ", round(time.time() - start,5))
    
    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw_img, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names_seq[label], score)
        draw_caption(draw_img, b, caption)
    
    if is_print:
        print("이미지 processing 시간: ", round(time.time() - start,5))
    
    return draw_img

# image 로드 
img = cv2.imread('../../data/image/john_wick01.jpg')
draw_img = get_detected_image_retina(retina_model, img, use_copied_array=True, is_print=True)
img_rgb = cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(14, 14))
plt.imshow(img_rgb)

In [ ]:
# Save cv2 image to json
import base64
import json
import pickle
from PIL import Image
def im2json(im):
    """Convert a Numpy array to a JSON string"""
    #_, imdata = cv2. imencode('.JPG', im)
    imdata = pickle.dumps(im)
    jstr = json.dumps({"image": base64.b64encode(imdata).decode('ascii')})
    return jstr
def json2im(jstr):
    """Convert a JSON string back to a Numpy array"""
    load = json.loads(jstr)
    imdata = base64.b64decode(load['image'])
    im = pickle.loads(imdata)
    return im

# Make img_rgb into JSON string
#jstr = im2json(img_rgb)

#OpenCVim = json2im (jstr)
#PILimage = Image.fromarray(OpenCVim[...,::-1])
#PILimage.show()

In [11]:
def detect_video_retina(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size= (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)
    
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break

        detected_image = get_detected_image_retina(model,image_frame, use_copied_array=False, is_print=True)
        vid_writer.write(detected_image)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [12]:
detect_video_retina(retina_model, '../../data/video/John_Wick_small.mp4', output_path="../../data/output/John_Wick_small_retina01.avi")

총 Frame 갯수: 58
object detection 처리 시간:  2.84229
이미지 processing 시간:  2.84362
object detection 처리 시간:  2.86916
이미지 processing 시간:  2.87047
object detection 처리 시간:  2.87631
이미지 processing 시간:  2.87742
object detection 처리 시간:  2.90749
이미지 processing 시간:  2.90902
object detection 처리 시간:  2.8947
이미지 processing 시간:  2.89594
object detection 처리 시간:  2.85351
이미지 processing 시간:  2.85481
object detection 처리 시간:  2.89677
이미지 processing 시간:  2.89838
object detection 처리 시간:  2.90268
이미지 processing 시간:  2.90416
object detection 처리 시간:  2.90232
이미지 processing 시간:  2.90377
object detection 처리 시간:  2.85647
이미지 processing 시간:  2.85786
object detection 처리 시간:  2.84114
이미지 processing 시간:  2.84238
object detection 처리 시간:  2.84132
이미지 processing 시간:  2.84242
object detection 처리 시간:  2.92556
이미지 processing 시간:  2.92719
object detection 처리 시간:  2.9253
이미지 processing 시간:  2.92704
object detection 처리 시간:  2.89641
이미지 processing 시간:  2.8987
object detection 처리 시간:  2.87192
이미지 processing 시간:  2.87357
object detec

In [13]:
!gsutil cp ../../data/output/John_Wick_small_retina01.avi gs://my_bucket_dlcv/data/output/John_Wick_small_retina01.avi

BucketNotFoundException: 404 gs://my_bucket_dlcv bucket does not exist.


In [ ]:
detect_video_retina(retina_model, '../../data/video/Night_Day_Chase.mp4', output_path="../../data/output/Night_Day_Chase_retina01.avi")

총 Frame 갯수: 1383
object detection 처리 시간:  2.79369
이미지 processing 시간:  2.79486
object detection 처리 시간:  2.80299
이미지 processing 시간:  2.80391
object detection 처리 시간:  2.81825
이미지 processing 시간:  2.81924
object detection 처리 시간:  2.75823
이미지 processing 시간:  2.75923
object detection 처리 시간:  2.82585
이미지 processing 시간:  2.82686
object detection 처리 시간:  2.78549
이미지 processing 시간:  2.78666
object detection 처리 시간:  2.80366
이미지 processing 시간:  2.8048
object detection 처리 시간:  2.80604
이미지 processing 시간:  2.80706
object detection 처리 시간:  2.80423
이미지 processing 시간:  2.80535
object detection 처리 시간:  2.79819
이미지 processing 시간:  2.79922
object detection 처리 시간:  2.79074
이미지 processing 시간:  2.79166
object detection 처리 시간:  2.78513
이미지 processing 시간:  2.78607
object detection 처리 시간:  2.77679
이미지 processing 시간:  2.77787
object detection 처리 시간:  2.76076
이미지 processing 시간:  2.76164
object detection 처리 시간:  2.80912
이미지 processing 시간:  2.81013
object detection 처리 시간:  2.76325
이미지 processing 시간:  2.76409
object d

object detection 처리 시간:  2.80154
이미지 processing 시간:  2.80262
object detection 처리 시간:  2.83847
이미지 processing 시간:  2.83949
object detection 처리 시간:  2.83116
이미지 processing 시간:  2.83211
object detection 처리 시간:  2.78636
이미지 processing 시간:  2.78736
object detection 처리 시간:  2.85092
이미지 processing 시간:  2.85186
object detection 처리 시간:  2.81312
이미지 processing 시간:  2.81405
object detection 처리 시간:  2.88304
이미지 processing 시간:  2.88421
object detection 처리 시간:  2.90661
이미지 processing 시간:  2.90789
object detection 처리 시간:  2.88872
이미지 processing 시간:  2.89008
object detection 처리 시간:  2.84182
이미지 processing 시간:  2.84327
object detection 처리 시간:  2.89326
이미지 processing 시간:  2.89402
object detection 처리 시간:  2.87448
이미지 processing 시간:  2.87535
object detection 처리 시간:  2.84445
이미지 processing 시간:  2.8455
object detection 처리 시간:  2.83194
이미지 processing 시간:  2.8328
object detection 처리 시간:  2.79385
이미지 processing 시간:  2.79468
object detection 처리 시간:  2.84308
이미지 processing 시간:  2.84421
object detection 처리 시간:  2

object detection 처리 시간:  2.88539
이미지 processing 시간:  2.88607
object detection 처리 시간:  2.90867
이미지 processing 시간:  2.90935
object detection 처리 시간:  2.92033
이미지 processing 시간:  2.92136
object detection 처리 시간:  2.87329
이미지 processing 시간:  2.87435
object detection 처리 시간:  2.90189
이미지 processing 시간:  2.90269
object detection 처리 시간:  2.88619
이미지 processing 시간:  2.88747
object detection 처리 시간:  2.94852
이미지 processing 시간:  2.9497
object detection 처리 시간:  2.90562
이미지 processing 시간:  2.90634
object detection 처리 시간:  2.87265
이미지 processing 시간:  2.87282
object detection 처리 시간:  2.87884
이미지 processing 시간:  2.87959
object detection 처리 시간:  2.90139
이미지 processing 시간:  2.90222
object detection 처리 시간:  2.89758
이미지 processing 시간:  2.89846
object detection 처리 시간:  2.92704
이미지 processing 시간:  2.92798
object detection 처리 시간:  2.88511
이미지 processing 시간:  2.88625
object detection 처리 시간:  2.91652
이미지 processing 시간:  2.91758
object detection 처리 시간:  2.91431
이미지 processing 시간:  2.91537
object detection 처리 시간:  

object detection 처리 시간:  2.82689
이미지 processing 시간:  2.82814
object detection 처리 시간:  2.80916
이미지 processing 시간:  2.81034
object detection 처리 시간:  2.7909
이미지 processing 시간:  2.79193
object detection 처리 시간:  2.84121
이미지 processing 시간:  2.84221
object detection 처리 시간:  2.80621
이미지 processing 시간:  2.80736
object detection 처리 시간:  2.80549
이미지 processing 시간:  2.80684
object detection 처리 시간:  2.81171
이미지 processing 시간:  2.81269
object detection 처리 시간:  2.84308
이미지 processing 시간:  2.8444
object detection 처리 시간:  2.8126
이미지 processing 시간:  2.81364
object detection 처리 시간:  2.87763
이미지 processing 시간:  2.87871
object detection 처리 시간:  2.81007
이미지 processing 시간:  2.81124
object detection 처리 시간:  2.82044
이미지 processing 시간:  2.82164
object detection 처리 시간:  2.79752
이미지 processing 시간:  2.79885
object detection 처리 시간:  2.82846
이미지 processing 시간:  2.8298
object detection 처리 시간:  2.83139
이미지 processing 시간:  2.83274
object detection 처리 시간:  2.84946
이미지 processing 시간:  2.85074
object detection 처리 시간:  2.8

object detection 처리 시간:  2.83569
이미지 processing 시간:  2.83692
object detection 처리 시간:  2.83139
이미지 processing 시간:  2.8324
object detection 처리 시간:  2.83731
이미지 processing 시간:  2.83831
object detection 처리 시간:  2.83795
이미지 processing 시간:  2.83915
object detection 처리 시간:  2.84
이미지 processing 시간:  2.84093
object detection 처리 시간:  2.83266
이미지 processing 시간:  2.83363
object detection 처리 시간:  2.83212
이미지 processing 시간:  2.83308
object detection 처리 시간:  2.84873
이미지 processing 시간:  2.84998
object detection 처리 시간:  2.86165
이미지 processing 시간:  2.86281
object detection 처리 시간:  2.85759
이미지 processing 시간:  2.85875
object detection 처리 시간:  2.83888
이미지 processing 시간:  2.83997
object detection 처리 시간:  2.85731
이미지 processing 시간:  2.8584
object detection 처리 시간:  2.84475
이미지 processing 시간:  2.84581
object detection 처리 시간:  2.81412
이미지 processing 시간:  2.81516
object detection 처리 시간:  2.86162
이미지 processing 시간:  2.86272
object detection 처리 시간:  2.84581
이미지 processing 시간:  2.84708
object detection 처리 시간:  2.89

object detection 처리 시간:  2.89534
이미지 processing 시간:  2.89629
object detection 처리 시간:  2.89476
이미지 processing 시간:  2.89565
object detection 처리 시간:  2.88596
이미지 processing 시간:  2.88661
object detection 처리 시간:  2.85389
이미지 processing 시간:  2.85495
object detection 처리 시간:  2.8425
이미지 processing 시간:  2.84357
object detection 처리 시간:  2.8543
이미지 processing 시간:  2.85527
object detection 처리 시간:  2.86601
이미지 processing 시간:  2.86691
object detection 처리 시간:  2.91311
이미지 processing 시간:  2.91405
object detection 처리 시간:  2.89835
이미지 processing 시간:  2.89955
object detection 처리 시간:  2.85569
이미지 processing 시간:  2.85657
object detection 처리 시간:  2.83434
이미지 processing 시간:  2.83516
object detection 처리 시간:  2.80115
이미지 processing 시간:  2.80208
object detection 처리 시간:  2.9143
이미지 processing 시간:  2.91502
object detection 처리 시간:  2.90352
이미지 processing 시간:  2.90453
object detection 처리 시간:  2.85983
이미지 processing 시간:  2.86099
object detection 처리 시간:  2.8562
이미지 processing 시간:  2.85737
object detection 처리 시간:  2.8

object detection 처리 시간:  2.90084
이미지 processing 시간:  2.90262
object detection 처리 시간:  2.8842
이미지 processing 시간:  2.88556
object detection 처리 시간:  2.85779
이미지 processing 시간:  2.85882
object detection 처리 시간:  2.88503
이미지 processing 시간:  2.88595
object detection 처리 시간:  2.87575
이미지 processing 시간:  2.87675
object detection 처리 시간:  2.85925
이미지 processing 시간:  2.86024
object detection 처리 시간:  2.87279
이미지 processing 시간:  2.8738
object detection 처리 시간:  2.84751
이미지 processing 시간:  2.84867
object detection 처리 시간:  2.87033
이미지 processing 시간:  2.87151
object detection 처리 시간:  2.84252
이미지 processing 시간:  2.8436
object detection 처리 시간:  2.87571
이미지 processing 시간:  2.87692
object detection 처리 시간:  2.8371
이미지 processing 시간:  2.83816
object detection 처리 시간:  2.89827
이미지 processing 시간:  2.89922
object detection 처리 시간:  2.87196
이미지 processing 시간:  2.87281
object detection 처리 시간:  2.86532
이미지 processing 시간:  2.86631
object detection 처리 시간:  2.89012
이미지 processing 시간:  2.89108
object detection 처리 시간:  2.9

In [ ]:
!gsutil cp ../../data/output/Night_Day_Chase_retina01.avi gs://my_bucket_dlcv/data/output/Night_Day_Chase_retina01.avi